<a href="https://colab.research.google.com/github/bhattacharya5/MLBD/blob/main/M21AIE211_MLBD_Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Write program in python but first download the MNIST dataset from https://www.kaggle.com/datasets/oddrationale/mnist-in-csv. 

Pick its training and test sets. Each sample in the
dataset is represented by a 784-dimensional vector. There are ten classes (the ten digits, `0’ to `9’), and each sample is associated with one class.


(1) Using the raw binary features, implement the streaming Naive Bayes algorithm and classify
the test data.

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd "/content/gdrive/My Drive/Colab Notebooks/semester3/MLBD"

/content/gdrive/My Drive/Colab Notebooks/semester3/MLBD


In [3]:
import numpy as np
import pandas as pd

In [4]:
df_train = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/semester3/MLBD/train.csv')
#df_train = pd.read_csv('/content/sample_data/train.csv')
df_train.head(5)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df_test = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/semester3/MLBD/test.csv')
#df_test = pd.read_csv('/content/sample_data/test.csv')
df_test.head(5)

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
Y_train = df_train["label"]

# Drop 'label' column
X_train = df_train.drop(labels = ["label"],axis = 1) 

X_test = df_test

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

In [32]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [33]:
# Implement the streaming Naive Bayes algorithm
class StreamingNaiveBayes:

    def __init__(self):
        self.classes = []
        self.priors = []
        self.conditional_probabilities = []

    def fit(self, X, y):
        # Count the number of instances in each class
        counts = np.unique(y, return_counts=True)[1]

        # Initialize the prior probabilities
        self.priors = counts / len(y)

        # Initialize the conditional probabilities
        for i in range(10):
            self.conditional_probabilities.append(np.zeros(784))

        # Update the prior probabilities and conditional probabilities
        for i in range(0, len(X), 1000):
            X_batch = X[i+1:i+1001]
            y_batch = y[i+1:i+1001]

            for j in range(len(X_batch)):
                x = X_batch[j]
                y_j = y_batch[j]

                # Update the prior probability for the current class
                self.priors[y_j] += 1

                # Update the conditional probability for the current class and feature
                for k in range(784):
                    self.conditional_probabilities[y_j][k] += x[k]

    def predict(self, X):
        predictions = []
        for x in X:
            # Calculate the probability of each class
            probabilities = []
            for i in range(10):
                probabilities.append(self.priors[i] * self.conditional_probabilities[i].dot(x))

            # Choose the class with the highest probability
            predictions.append(np.argmax(probabilities))

        return predictions

# Train the streaming Naive Bayes algorithm
model = StreamingNaiveBayes()
model.fit(X_train, y_train)

# Predict the labels of the test data
y_pred = model.predict(X_test)

# Calculate the accuracy
accuracy = np.mean(y_pred == y_test)
print('Accuracy:', accuracy)


Accuracy: 0.6780952380952381
